In [212]:
import random
import numpy as np

class Neuron:
    def __init__(self, numberOfOutgoing):
        self.activation = random.uniform(0, 1)
        self.bias = random.uniform(0, 1)
        self.outgoingWeights = np.array([random.uniform(0, 1) for i in range(numberOfOutgoing)])

    def cout(self):
        print("Activation: ", self.activation)
        print("Bias: ", self.bias)
        print("Outgoing Weights: ", self.outgoingWeights)


In [213]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class Network:

    def __init__(self, start, first, second, end):
        self.startLayer = np.array([Neuron(first) for i in range(start)])
        self.firstLayer = np.array([Neuron(second) for i in range(first)])
        self.secondLayer = np.array([Neuron(end) for i in range(second)])
        self.endLayer = np.array([Neuron(0) for i in range(end)])

    def forwardPropagationStep(self, inLayer, outLayer):
        # weight matrix
        weightMatrix = np.array([inLayer[0].outgoingWeights])
        for i in range(1, outLayer.size):
            weightMatrix = np.r_[weightMatrix, [inLayer[i].outgoingWeights]]

        # input activation
        inActivation = np.array([neur.activation for neur in inLayer])
        
        # output biases
        outBias = np.array([neur.bias for neur in outLayer])

        outputMat = np.dot(weightMatrix, inActivation)
        outputMat = outputMat - outBias

        for i in range(outLayer.size):
            outLayer[i].activation = sigmoid(outputMat[i])

    def fullForwardPropagation(self):
        self.forwardPropagationStep(self.startLayer, self.firstLayer)
        self.forwardPropagationStep(self.firstLayer, self.secondLayer)
        self.forwardPropagationStep(self.secondLayer, self.endLayer)
        print(self.cost(1))
    
    def cost(self, target):
        sum = 0
        for i in range(self.endLayer.size):
            if (i == target):
                sum += pow(self.endLayer[i].activation - 1.0, 2)
            else:
                sum += pow(self.endLayer[i].activation, 2)
        return sum


    def cout(self):
        print("Start Layer")
        for neur in self.startLayer:
            neur.cout()
        print("First Layer")
        for neur in self.firstLayer:
            neur.cout()
        print("Second Layer")
        for neur in self.secondLayer:
            neur.cout()
        print("End Layer")
        for neur in self.endLayer:
            neur.cout()
        

In [214]:
net = Network(2, 2, 2, 2)
net.fullForwardPropagation()

NameError: name 'endLayer' is not defined